## クラスタ分析

### 卸売業者のデータ分析

クラスタリングの利用法を学ぶために、Wholesale customers Data Setを使用します。

In [16]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.cluster import KMeans
df = pd.read_csv('Wholesale customers data (3).csv',header = 1)

In [17]:
df_ = df.loc[:,['Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen']]

In [3]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca = pca.fit(df_)
data_pca = pca.transform(df_)

In [26]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA(n_components=None)
pca = pca.fit(df_)
var_exp = pca.explained_variance_ratio_
cum_var_exp = np.cumsum(var_exp)
print(cum_var_exp)
# 個々の説明された分散
plt.bar(range(1,7), var_exp, alpha=0.5, align='center', label='individual explained variance')
# 累積説明分散
plt.step(range(1,7), cum_var_exp, where='mid', label='cumulative expalined variance')
plt.ylabel('Explained variance ratio') #説明された分散比
plt.xlabel('Principal components') #主成分
plt.hlines(0.7, 0, 6,  "blue", linestyles='dashed')
plt.legend(loc='best')
plt.grid()
plt.show()

## 【問題10】クラスタ数kの選択

### エルボー法

In [24]:
SSE=[]
for i in range(1,8):
    model = KMeans(n_clusters=i,
               init='k-means++',
               n_init=5,
               max_iter=10,
               random_state=0)
    model.fit(data_pca)
    SSE.append(model.inertia_)
plt.plot(range(1,8), SSE, marker='o')
plt.xticks(np.arange(1,8,1))
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

### 選択と説明 

- 最適だと思われるクラスタの数は３個だと思います。

- ４個めを設置するとエルボー法の傾きが緩やかになるため、３を選択しました。

## 【問題11】既知のグループとの比較

### Region

In [6]:
from sklearn.cluster import KMeans

In [7]:
df_r = df.loc[:,['Region']]
df_r_np = np.array(df_r)
df_r_np = df_r_np.reshape(1,-1)
skm1 = KMeans(n_clusters=3)
skm1.fit(data_pca)
pca_pred = skm1.predict(data_pca)

In [23]:
plt.xlabel("kurasuta")
plt.ylabel("Region")
plt.title("Region")
plt.hist(pca_pred,stacked = True,alpha = 0.5)
plt.hist(df_r_np,stacked = True,alpha = 0.5)
#plt.legend()
plt.show()

### Channel

In [18]:
df_c = df.loc[:,['Channel']]
df_c_np = np.array(df_c)
df_c_np = df_c_np.reshape(1,-1)
skm2 = KMeans(n_clusters=3)
skm2.fit(data_pca)
pca_pred = skm1.predict(data_pca)

In [22]:
plt.xlabel("kurasuta")
plt.ylabel("Channel")
plt.title("Channel")
plt.hist(pca_pred,stacked = True,alpha = 0.5)
plt.hist(df_c_np,stacked = True,alpha = 0.5)

plt.show()

## 違い

- RegionとChannelのクラスタとのグラフの対比を見て貰えばわかる通り、クラスタの割り当てに相違があることが見受けられるので、比較の対象には出来ない。

## 【問題12】卸売業者にとって有益な情報

In [19]:
df.info()

In [20]:
df.describe()

In [21]:
df.corr()

In [25]:
import seaborn as sns; 
sns.set(color_codes=True)
corrMatt = df.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

- 考察

色々な情報を可視化した結果、ヒートマップの情報が有力なものと見受けられ、GroceryとDetergents_Paperの相関関係が強く、次いでGroceryとMilkです。
この結果から、Groceryと関係性の深いDetergents_Paper、及びMilkに着目した経営戦略を展開するのが望ましい。